# 8) mnist 데이터셋(skorch)

In [1]:
# skorch : pytorch를 쉽게 구현할 수 있는 라이브러리

from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
X_train= X_train.reshape(-1,784)        # -1 : 행은 가변적으로 사용
X_test= X_test.reshape(-1,784)

X_train=X_train/255.
X_test=X_test/255.

In [ ]:
#   numpy() : 메모리
#   .cpu() : cpu 
#   .cuda() : gpu

In [3]:
import torch

X_train=torch.from_numpy(X_train).float()
y_train=torch.from_numpy(y_train.astype('int32')).long()

X_test=torch.from_numpy(X_test).float()
y_test=torch.from_numpy(y_test.astype('int32')).long()

print(X_train.shape)
print(X_test.shape)

X_train=X_train.cuda()
y_train=y_train.cuda()

X_test=X_test.cuda()
y_test=y_test.cuda()

torch.Size([60000, 784])
torch.Size([10000, 784])


In [5]:
import torch.nn as nn
import torch.nn.functional as F

# 신경망 구성
class Net(nn.Module):
#         상위클래스
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)

model=Net().cuda()
#     클래스 => 메모리, 인스턴스화 => 초기화 함수(__init__) 호출

In [8]:
# pip install skorch

from skorch import NeuralNetClassifier

net = NeuralNetClassifier(Net,max_epochs=20,lr=0.1)
#                         모형      에폭     학습률

net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6422       0.9002        0.3481  2.0762
      2        0.3115       0.9188        0.2825  3.0089
      3        0.2591       0.9296        0.2436  2.6095
      4        0.2235       0.9383        0.2153  2.7566
      5        0.1968       0.9443        0.1938  1.6750
      6        0.1758       0.9501        0.1772  1.6665
      7        0.1588       0.9529        0.1640  1.6144
      8        0.1447       0.9552        0.1538  1.7496
      9        0.1327       0.9575        0.1453  1.6590
     10        0.1225       0.9597        0.1381  1.8068
     11        0.1136       0.9613        0.1321  1.6898
     12        0.1058       0.9623        0.1268  1.5224
     13        0.0988       0.9631        0.1221  1.6268
     14        0.0927       0.9643        0.1180  1.4910
     15        0.0871       0.9657        0.1144  1.5047
     16        0.0820       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=784, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  ),
)

In [9]:
import numpy as np

# 모형의 정확도 계산

pred = net.predict(X_test)
y_test = y_test.cpu().numpy()

accuracy = np.mean(pred == y_test)
accuracy

0.9731

In [10]:
from torchinfo import summary

summary(model, input_size=(100,784))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83